In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()


True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH05-Memory")


LangSmith 추적을 시작합니다.
[프로젝트명]
CH05-Memory


In [3]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# 단계 1: 문서 로드(Load Documents)
from langchain.document_loaders import MongodbLoader
from server.db import DB
from server.db import get_mongo_connection_string

channel_id = 1890652954

loader = MongodbLoader(
    connection_string=get_mongo_connection_string(),
    db_name=DB.NAME,
    collection_name=DB.COLLECTION.CHANNEL.DATA,
    filter_criteria={"channelId": channel_id}, # 데이터베이스에서 조회할 기준 (쿼리)
    field_names=("text",),
    metadata_names=("id", "timestamp", "channelId", "views", "url"), # 메타데이터로 지정할 필드 목록
)

import nest_asyncio
nest_asyncio.apply()  # Jupyter Notebook에서 asyncio 실행 문제 해결

docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
splitted_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embedding = OpenAIEmbeddings()

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성한다.
vectorstore = FAISS.from_documents(documents=splitted_documents, embedding=embedding)

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성한다.
retriever = vectorstore.as_retriever(search_kwargs={"k": 6})

In [15]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


# 프롬프트 정의
prompt = PromptTemplate.from_template(
    """You are an assistant responding to inquiries from an investigator trying to investigate a drug-selling channel. 
Below is the chat data (context) collected from a Telegram channel where drugs are sold, along with our previous Q&A Chat history. Based on this context chat data and previous chat history, answer questions about the transaction details of the channel.
If you don't know the answer, just say that you don't know.
Answer in Korean.

#Context: 
{context} 

#Our Previous Chat History:
{chat_history}

#Question: 
{question} 

#Answer:""")

# llm 생성
llm = ChatOpenAI(model_name="gpt-4o")

# 일반 Chain 생성
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "chat_history": itemgetter("chat_history"),
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [5]:
# 세션 기록을 저장할 딕셔너리
store = {}


# 세션 ID를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    print(f"[대화 세션ID]: {session_ids}")
    if session_ids not in store:  # 세션 ID가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="question",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="chat_history",  # 기록 메시지의 키
)

chain_with_history.invoke(
    # 질문 입력
    {"question": "이 채널에서 판매되는 마약의 지역을 말해줘."},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "abc123"}},
)

[대화 세션ID]: abc123


'이 채널에서는 부산, 창원, 울산, 서울, 광주, 천안을 포함한 수도권 지역에서 마약이 판매되는 것으로 보입니다.'

In [6]:
chain_with_history.invoke(
    # 질문 입력
    {"question": "내가 이전에 물어본 내용과 그 답변이 뭐였지?", "context": retriever},
    # 세션 ID 기준으로 대화를 기록합니다.
    config={"configurable": {"session_id": "abc123"}},
)

[대화 세션ID]: abc123


'당신이 이전에 물어본 내용은 "이 채널에서 판매되는 마약의 지역을 말해줘."였으며, 그에 대한 답변은 "이 채널에서는 부산, 창원, 울산, 서울, 광주, 천안을 포함한 수도권 지역에서 마약이 판매되는 것으로 보입니다."였습니다.'

In [16]:
# TODO: https://wikidocs.net/233813 를 참고해서 SQLite에 대화내역 오프라인 저장하고 불러오기

In [17]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

# SQLChatMessageHistory 객체를 생성하고 세션 ID와 데이터베이스 연결 파일을 설정
chat_message_history = SQLChatMessageHistory(
    session_id="sql_history", connection="sqlite:///sqlite.db"
)


In [18]:
# 사용자 메시지를 추가합니다.
chat_message_history.add_user_message(
    "안녕? 만나서 반가워. 내 이름은 테디야. 나는 랭체인 개발자야. 앞으로 잘 부탁해!"
)
# AI 메시지를 추가합니다.
chat_message_history.add_ai_message("안녕 테디, 만나서 반가워. 나도 잘 부탁해!")


In [19]:
# 채팅 메시지 기록의 메시지들
chat_message_history.messages

[HumanMessage(content='안녕? 만나서 반가워. 내 이름은 테디야. 나는 랭체인 개발자야. 앞으로 잘 부탁해!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕 테디, 만나서 반가워. 나도 잘 부탁해!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='안녕? 만나서 반가워. 내 이름은 테디야. 나는 랭체인 개발자야. 앞으로 잘 부탁해!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕 테디, 만나서 반가워. 나도 잘 부탁해!', additional_kwargs={}, response_metadata={})]

In [23]:
def get_chat_history(user_id, conversation_id):
    return SQLChatMessageHistory(
        table_name=user_id,
        session_id=conversation_id,
        connection="sqlite:///sqlite.db",
    )

from langchain_core.runnables.utils import ConfigurableFieldSpec

config_fields = [
    ConfigurableFieldSpec(
        id="user_id",
        annotation=str,
        name="User ID",
        description="Unique identifier for a user.",
        default="",
        is_shared=True,
    ),
    ConfigurableFieldSpec(
        id="conversation_id",
        annotation=str,
        name="Conversation ID",
        description="Unique identifier for a conversation.",
        default="",
        is_shared=True,
    ),
]

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,  # 대화 기록을 가져오는 함수를 설정합니다.
    input_messages_key="question",  # 입력 메시지의 키를 "question"으로 설정
    history_messages_key="chat_history",  # 대화 기록 메시지의 키를 "history"로 설정
    history_factory_config=config_fields,  # 대화 기록 조회시 참고할 파라미터를 설정합니다.
)

# config 설정
config = {"configurable": {"user_id": "user1", "conversation_id": "conversation1"}}


In [21]:
# 질문과 config 를 전달하여 실행합니다.
chain_with_history.invoke({"question": "안녕 반가워, 내 이름은 테디야"}, config)

'죄송합니다만, 제공된 채팅 데이터 및 이전 대화 기록으로는 해당 텔레그램 채널의 거래 세부 사항, 약물의 종류, 가격 등 구체적인 정보를 확인할 수 없습니다. 추가적인 구체적인 데이터나 증거가 필요할 것 같습니다.'

In [22]:
# 후속 질문을 실해합니다.
chain_with_history.invoke({"question": "내 이름이 뭐라고?"}, config)


'제공된 채팅 데이터에서는 귀하의 이름을 알 수 있는 정보가 없습니다.'